In [12]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
from sklearn.metrics.pairwise import cosine_similarity
import csv
from scipy.sparse import csr_matrix

In [13]:
# read numerical reference list
with open('paper_numbering_ref.csv', mode ='r',encoding='utf-8')as file:
    csvFile = csv.reader(file)
    ref_dict = {}
    next(csvFile)
    for i in csvFile:
        ref_dict[i[0]] = list(i[1].split())

In [14]:
# Finding out how many distinct papers there are
with open('numbering_ref.csv', mode ='r',encoding='utf-8')as file:
    final_line = file.readlines()[-1]
    total_ref_num = int(final_line.split(',')[0])+1

In [15]:
id_list = list(ref_dict.keys())
total_paper_num=len(id_list)

In [16]:
client = MongoClient('mongodb://localhost:27017')
db = client['Paper']
collection = db['Papers']

In [17]:
# Creating components of sparse matrix which holds distribution of references each paper
row = []
column = []
data = []
count = 0
for i in ref_dict:
    for j in range(len(ref_dict[i])):
        row.append(count)
        column.append(int(ref_dict[i][j]))
        data.append(1)
    count = count + 1

In [18]:
# creating sparse matrix
sparseMatrix = csr_matrix((data, (row, column)), 
                          shape = (total_paper_num, total_ref_num))

In [19]:
# Calculate reference similarity between papers
matrix = cosine_similarity(sparseMatrix)

In [20]:
def give_rec(title, matrix=matrix):
    
    data = collection.find({'title' : title})
    
    # Get the paper id corresponding to title
    paper_id = data[0]['id']
    
    # Get the index corresponding to paper_id
    idx = id_list.index(paper_id)

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(matrix[idx]))

    # Sort the paper 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    
    top_n_list = []
    
    for i in range(1,11):
        paper_id = id_list[sig_scores[i][0]]
        data = collection.find({'id' : str(paper_id)})
        top_n_list.append([paper_id,data[0]['title'],sig_scores[i][1]])

    df = pd.DataFrame(top_n_list, columns = ["id", "title", "score"])
    return df

In [21]:
top_n_ref_df = give_rec('Semi-Supervised Learning with Ladder Networks')

In [22]:
top_n_ref_df.head(10)

,id,title,score
0,1504.08215v1,Lateral Connections in Denoising Autoencoders ...,0.571187
1,1611.02320v1,Adversarial Ladder Networks,0.328798
2,1803.01216v1,Deep Bayesian Active Semi-Supervised Learning,0.279108
3,1612.01942v1,Semi-Supervised Learning with the Deep Renderi...,0.226294
4,1602.02389v4,Ensemble Robustness and Generalization of Stoc...,0.206689
5,1711.06756v1,Deep supervised learning using local errors,0.200932
6,1506.02351v8,Stacked What-Where Auto-encoders,0.198273
7,1412.6544v6,Qualitatively characterizing neural network op...,0.188579
8,1611.07119v1,Max-Margin Deep Generative Models for (Semi-)S...,0.179427
9,1412.7210v4,Denoising autoencoder with modulated lateral c...,0.171398
